In [ ]:
import requests
import pandas as pd
import os,errno
import sys
import time
import numpy as np
import xarray as xr
import datetime as dt
import pandas as pd
import glob

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
yrs=np.arange(2023,2024)
month_days=[31,28,31,30,31,30,31,31,30,31,30,31]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    month_days[1]=28
    if year%4==0:
        month_days[1]=29
        
    for j in range(1,13,1):
        if j<10:
            month_str='0'+str(j)
        else:
            month_str=str(j)
            
        for k in range(month_days[j-1]):
            if k+1<10:
                day_str='0'+str(k+1)
            else:
                day_str=str(k+1)
            
            print(str(year)+month_str+day_str)
            
            files=glob.glob(dir_data+'MERRA2_400*'+str(year)+month_str+day_str+'*.nc')
            if len(files)>1:
                print('ERROR')
                print(files)
                sys.exit()
            
            ds=xr.open_dataset(files[0])
            #iwv=ds['TQV'][0::3,:,:]
            iwv=ds['U'][0::2,:,:]
            
            
            #ivtu=ds['UFLXQV'][0::3,:,:]
            #ivtv=ds['VFLXQV'][0::3,:,:]
            #ivt=np.sqrt(ivtu**2+ivtv**2)
            
            #mfc=ds['DQVDT_DYN'][0::3,:,:]
            
            if j==1 and k==0:
                iwv_concat=iwv
            else:
                iwv_concat=xr.concat([iwv_concat,iwv],dim='time')

    iwv_concat=iwv_concat.sel(lev=850)
    dk=xr.Dataset()
    dk['u']=(('time','lat','lon'),iwv_concat.values)

    dk.coords['time']=iwv_concat.time
    dk.coords['lat']=iwv_concat.lat.values
    dk.coords['lon']=iwv_concat.lon.values
    dk['u'].attrs["units"]='m*s-1'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    
    try:
        os.remove(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir3+'merra2.u850.6hr.'+str(year)+'.nc',mode='w',format='NETCDF4')
    
    #for f in files:
    #    os.remove(f)